In [15]:
import pandas as pd
import random

#S_Box
s_box = [0x5, 0x3, 0xA, 0x1, 0xE, 0xD, 0x2, 0x9, 0x6, 0xF, 0xC, 0x7, 0xB, 0x8, 0x0, 0x4]



In [16]:
#DDT
ddt = [[0 for _ in range(16)] for _ in range(16)]

for x1 in range(16):
    for x2 in range(16):
        delta_x = x1 ^ x2
        delta_y = s_box[x1] ^ s_box[x2]
        ddt[delta_x][delta_y] += 1


ddt_df = pd.DataFrame(ddt)
print(ddt_df)

    0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0   16   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
1    0   0   0   4   2   0   2   0   0   2   0   6   0   0   0   0
2    0   0   2   0   2   0   0   0   2   0   2   2   4   0   0   2
3    0   2   0   2   2   0   0   2   2   2   0   0   0   0   0   4
4    0   0   0   2   0   0   0   2   4   0   0   2   2   2   2   0
5    0   0   0   4   2   0   0   2   4   0   0   0   0   2   2   0
6    0   0   0   0   2   0   2   4   0   0   2   2   2   0   0   2
7    0   2   2   0   2   0   0   2   0   0   0   0   4   0   0   4
8    0   0   2   2   0   4   4   0   0   0   0   0   2   2   0   0
9    0   0   0   0   0   2   6   0   0   2   2   0   0   4   0   0
10   0   2   0   0   2   0   0   0   0   6   0   0   2   0   4   0
11   0   0   4   0   0   2   0   2   0   0   4   0   0   2   0   2
12   0   0   2   0   0   2   0   0   2   0   2   2   0   0   6   0
13   0   4   0   0   0   4   0   0   2   0   0   2   0   2   2

In [17]:
#generate plaintext
plaintexts = [random.getrandbits(16) for _ in range(10000)]

#save the plaintext in txt file
with open("plaintexts.txt", "w") as file:
    for pt in plaintexts:

        file.write(f"{hex(pt)}\n")

In [18]:
#key 
keys = [0x30cf, 0x87e3, 0xdcff, 0xcd6, 0x43c7]


In [ ]:

def permute(pt):
    temp = [(pt >> i) & 1 for i in reversed(range(16))]
    pattern = [0,4,8,12,1,5,9,13,2,6,10,14,3,7,11,15]
    new_pt = [0]*16
    for i in range(16):
        new_pt[pattern[i]] = temp[i]

    result_pt = 0
    for i, bit in enumerate(new_pt):
        result_pt |= (bit << (15 - i))
    return result_pt



def substitute(pt):
    block1 = (pt >> 12) & 0xF  
    block2 = (pt >> 8) & 0xF   
    block3 = (pt >> 4) & 0xF   
    block4 = pt & 0xF          
    new_block1 = s_box[block1]
    new_block2 = s_box[block2]
    new_block3 = s_box[block3] 
    new_block4 = s_box[block4]

    substituted_block = (new_block1 << 12) | (new_block2 << 8) | (new_block3 << 4) | new_block4
    return substituted_block



def encrypt(pt, keys):
    for k in range(3):
        pt ^= keys[k]
        pt = substitute(pt)
        pt = permute(pt)
    
    pt ^= keys[3]
    pt = substitute(pt)
    pt ^= keys[4]
    return pt


In [20]:
#encrypt the plaintext
ciphertexts = [encrypt(pt, keys) for pt in plaintexts]

In [ ]:
#save the cipherext in txt file
with open("ciphertexts.txt", "w") as file:
    for ct in ciphertexts:
        file.write(f"{hex(ct)}\n")

